# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd
import numpy as np
import gmaps
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sinabang,2.4803,96.3801,80.98,79,98,2.28,ID,1690417900
1,1,gondal,21.9667,70.8000,77.41,94,100,9.84,IN,1690417925
2,2,port-aux-francais,-49.3500,70.2167,40.03,77,90,38.70,TF,1690417925
3,3,mermoz boabab,14.7065,-17.4758,83.98,79,0,0.00,SN,1690417925
4,4,avarua,-21.2078,-159.7750,75.25,64,100,8.05,CK,1690417925


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
gmaps.configure(api_key=geoapify_key)
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"].astype(float)

# Display the map
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=humidity,
    dissipating=False, max_intensity=5,
    point_radius=1
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
new_types_df = pd.DataFrame(city_data_df, columns = ["City", "Country", "Lat", "Lng", "Max Temp", "Wind Speed", "Cloudiness"])
new_types_df = new_types_df.loc[(new_types_df["Max Temp"] < 80) & (new_types_df["Max Temp"] > 60)]
new_types_df = new_types_df.loc[new_types_df["Wind Speed"] < 10]
new_types_df = new_types_df.loc[new_types_df["Cloudiness"] == 0]

# Drop any rows with null values
new_types_df.dropna(inplace=True)

# Display sample data
new_types_df

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness
7,uruacu,BR,-14.5247,-49.1408,74.43,3.02,0
14,kone,NC,-21.0595,164.8658,75.38,8.88,0
38,palmas,BR,-10.2128,-48.3603,75.07,3.44,0
46,colonia,DE,50.9333,6.9500,63.97,9.22,0
68,beyneu,KZ,45.3167,55.2000,78.64,9.80,0
102,yeraliyev,KZ,43.2000,51.6500,75.67,7.29,0
161,itacoatiara,BR,-3.1431,-58.4442,79.18,0.63,0
193,fes,MA,34.0372,-4.9998,71.85,3.44,0
217,bermeo,ES,43.4209,-2.7215,67.24,4.23,0
267,melouza,DZ,35.9800,4.1867,73.09,6.08,0


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
#hotel_df = new_types_df
hotel_df = pd.DataFrame(new_types_df, columns=["City", "Country", "Lat", "Lng", "Max Temp", "Wind Speed", "Cloudiness"]).copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Hotel Name
7,uruacu,BR,-14.5247,-49.1408,74.43,3.02,0,
14,kone,NC,-21.0595,164.8658,75.38,8.88,0,
38,palmas,BR,-10.2128,-48.3603,75.07,3.44,0,
46,colonia,DE,50.9333,6.9500,63.97,9.22,0,
68,beyneu,KZ,45.3167,55.2000,78.64,9.80,0,
102,yeraliyev,KZ,43.2000,51.6500,75.67,7.29,0,
161,itacoatiara,BR,-3.1431,-58.4442,79.18,0.63,0,
193,fes,MA,34.0372,-4.9998,71.85,3.44,0,
217,bermeo,ES,43.4209,-2.7215,67.24,4.23,0,
267,melouza,DZ,35.9800,4.1867,73.09,6.08,0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set hotel variables
#target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
radius = 10000
# set up a parameters
params = {'categories':'accommodation.hotel',\
          'filter':'',\
          'bias':'',\
          'apiKey':geoapify_key,\
          'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
        
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
        # Log the search results
print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
la rinconada - nearest hotel: Hotel Torre de los Guzmanes


,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Hotel Name
7,uruacu,BR,-14.5247,-49.1408,74.43,3.02,0,No hotel found
14,kone,NC,-21.0595,164.8658,75.38,8.88,0,Pacifik Appartels
38,palmas,BR,-10.2128,-48.3603,75.07,3.44,0,Hotel Triângulo Mineiro
46,colonia,DE,50.9333,6.9500,63.97,9.22,0,Wasserturm Hotel Cologne
68,beyneu,KZ,45.3167,55.2000,78.64,9.80,0,Johan yu
102,yeraliyev,KZ,43.2000,51.6500,75.67,7.29,0,Kuryk
161,itacoatiara,BR,-3.1431,-58.4442,79.18,0.63,0,Regine's Hotel
193,fes,MA,34.0372,-4.9998,71.85,3.44,0,Maghreb Hôtel
217,bermeo,ES,43.4209,-2.7215,67.24,4.23,0,Aldatzeta ostatua
267,melouza,DZ,35.9800,4.1867,73.09,6.08,0,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
%%capture --no-display

# Configure the map plot
hotel_df['Hotel Name'] = hotel_info
hotel_df.dropna()
hotel_df.to_csv('../output_data/hotel_export.csv')

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# Display the map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

AttributeError: module 'collections' has no attribute 'Iterable'